In [1]:
from bluesky.network.client import Client
import os
from bluesky.network.client import Client
import google.generativeai as genai
from dotenv import load_dotenv, find_dotenv
import sys
from contextlib import contextmanager
from io import StringIO
from openai import OpenAI


Using Python-based geo functions


c:\Users\justa\OneDrive\Desktop\Developer\Thesis_ATM_with_LLM\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(find_dotenv())
# GOOGLE_API_KEY = os.environ.get('GEMINI_API_KEY')
# genai.configure(api_key=GOOGLE_API_KEY)
# model = genai.GenerativeModel('gemini-pro')
# generation_config = genai.GenerationConfig(
#     temperature=0.9,
#     candidate_count=1
# )
# response = model.generate_content(
#     prompt, generation_config=generation_config)
# txt_answer = response.text

gpt35 = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

In [3]:




@contextmanager
def capture_stdout():
    new_stdout = StringIO()
    old_stdout = sys.stdout
    sys.stdout = new_stdout
    try:
        yield new_stdout
    finally:
        sys.stdout = old_stdout


def update_until_complete(client):
    complete_output = ""
    while True:
        with capture_stdout() as captured:
            client.update()
        new_output = captured.getvalue()
        if not new_output.strip():
            # No more new output, break the loop
            break
        complete_output += new_output
    return complete_output

In [4]:
# a function to get the response from the model and return it. Input is the prompt
# def get_response(prompt):
#     response = model.generate_content(
#         prompt, generation_config=generation_config)
#     txt_answer = response.text
#     return txt_answer

# a function to get the response from the model and return it. Input is the prompt. model openai gpt

# completion = gpt35.chat.completions.create(
#     model="gpt-3.5-turbo-0613",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": "Hello!"}
#     ]
# )

# print(completion.choices[0].message.content)

def get_response(prompt):
    completion = gpt35.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content




In [5]:
client = Client()
client.connect("127.0.0.1", 11000, 11001)
client.update()

Client b'\x00\r*\xb3g' connected to host b'\x00m@\xeb\xb9'
Client active node changed.


In [6]:
client.update()

In [33]:
# command = 'CRE BBB999 A320 -15.101549,-14.504519 48 FL150 150'
command = 'CRE BBB999 A320 -15.101549,-14.504519 48 FL150 150'
command = 'GETACIDS'
client.send_event(b'STACK', command)

In [37]:
sim_output = update_until_complete(client)
print("All captured output:\n", sim_output)
print('-------------------')

All captured output:
 
-------------------


In [35]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [36]:
from langchain.agents import tool

@tool
def get_all_aircraft_info(command: str) -> str:
    """Returns each aircraft information. Use command GETACIDS to get all aircrafts."""
    client.send_event(b'STACK', command)
    sim_output = update_until_complete(client)
    return sim_output


In [38]:
tools = [get_all_aircraft_info]

In [39]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, that can also use tools to answer questions",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [40]:
llm_with_tools = llm.bind_tools(tools)

In [41]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [42]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [45]:
out_lst = list(agent_executor.stream(
    {"input": "Which aircraft has the lowest altitude?"}))



> Entering new AgentExecutor chain...

Invoking: `get_all_aircraft_info` with `{'command': 'GETACIDS'}`


KL102
Aircraft ids: ['KL101', 'KL102', 'KL103', 'BBB999']
Info on KL101 B74F index = 0
Pos: N51'15'45.0"  E006'00'0.0"
Hdg: 180   Trk: 180
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805
LNAV to KL101001

Info on KL102 B74F index = 1
Pos: N51'15'15.0"  E006'04'11.0"
Hdg: 210   Trk: 210
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805
LNAV to KL102001

Info on KL103 B74F index = 2
Pos: N51'13'22.0"  E006'07'49.0"
Hdg: 240   Trk: 240
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805
LNAV to KL103001

Info on BBB999 A320 index = 3
Pos: S15'01'8.0"  W014'24'34.0"
Hdg: 048   Trk: 048
Alt: 15000 ft  V/S: 0 fpm
CAS/TAS/GS: 150/188/188 kts   M: 0.300

The aircraft with the lowest altitude is BBB999 (A320) at 15000 ft.

> Finished chain.


In [47]:
list(agent_executor.stream(
    {"input": "So what is KL101 position?"}))



> Entering new AgentExecutor chain...

Invoking: `get_all_aircraft_info` with `{'command': 'GETACIDS'}`


KL102
Aircraft ids: ['KL101', 'KL102', 'KL103', 'BBB999']
Info on KL101 B74F index = 0
Pos: N51'10'51.0"  E006'00'0.0"
Hdg: 180   Trk: 180
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805
LNAV to KL101001

Info on KL102 B74F index = 1
Pos: N51'11'1.0"  E006'00'13.0"
Hdg: 210   Trk: 210
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805
LNAV to KL102001

Info on KL103 B74F index = 2
Pos: N51'10'56.0"  E006'01'2.0"
Hdg: 240   Trk: 240
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805
LNAV to KL103001

Info on BBB999 A320 index = 3
Pos: S14'59'51.0"  W014'23'6.0"
Hdg: 048   Trk: 048
Alt: 15000 ft  V/S: 0 fpm
CAS/TAS/GS: 150/188/188 kts   M: 0.300

The position of aircraft KL101 is N51'10'51.0" E006'00'0.0". It is flying at an altitude of 25000 ft with a heading and track of 180 degrees. The speed indicators are as follows: CAS 340 kts, TA

[{'actions': [OpenAIToolAgentAction(tool='get_all_aircraft_info', tool_input={'command': 'GETACIDS'}, log="\nInvoking: `get_all_aircraft_info` with `{'command': 'GETACIDS'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gROnBM70gLRS0MoF7zc6Yri5', 'function': {'arguments': '{"command":"GETACIDS"}', 'name': 'get_all_aircraft_info'}, 'type': 'function'}]})], tool_call_id='call_gROnBM70gLRS0MoF7zc6Yri5')],
  'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gROnBM70gLRS0MoF7zc6Yri5', 'function': {'arguments': '{"command":"GETACIDS"}', 'name': 'get_all_aircraft_info'}, 'type': 'function'}]})]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='get_all_aircraft_info', tool_input={'command': 'GETACIDS'}, log="\nInvoking: `get_all_aircraft_info` with `{'command': 'GETACIDS'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gROnBM70gLRS0MoF7zc6Yri5', 'function': {'argumen

In [ ]:
# TODO
# 1. connect to a memory block so it will be able to remember the previous commands and responses
# 2. create a vector database with commands and their descriptions (just like voyager did). Input is a prompt, output is top 5 commands
# 3. Make a system that makes the agent write its own commands and test out the commands



In [48]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [77]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, SystemMessage

chat_history = []

In [72]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [111]:
chat_history = []
input1 = "how many aircraft are there in the sky currently?"

result1 = agent_executor.invoke(

    {"input": input1, "chat_history": chat_history})

# chat history should have human message, output result from sim, and output result from openai



> Entering new AgentExecutor chain...

Invoking: `get_all_aircraft_info` with `{'command': 'GETACIDS'}`


KL102
Aircraft ids: ['KL101', 'KL102', 'KL103', 'BBB999']
Info on KL101 B74F index = 0
Pos: N49'04'7.0"  E006'00'2.0"
Hdg: 180   Trk: 180
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805

Info on KL102 B74F index = 1
Pos: N49'21'14.0"  E004'21'9.0"
Hdg: 210   Trk: 210
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805

Info on KL103 B74F index = 2
Pos: N50'06'37.0"  E003'08'47.0"
Hdg: 239   Trk: 239
Alt: 25000 ft  V/S: 0 fpm
CAS/TAS/GS: 340/485/485 kts   M: 0.805

Info on BBB999 A320 index = 3
Pos: S14'26'56.0"  W013'45'17.0"
Hdg: 048   Trk: 048
Alt: 15000 ft  V/S: 0 fpm
CAS/TAS/GS: 150/188/188 kts   M: 0.300

There are currently 4 aircraft in the sky. The aircraft IDs are: KL101, KL102, KL103, and BBB999.

> Finished chain.


In [112]:
intermediate_steps1 = result1['intermediate_steps'][0][1]

chat_history.extend(
    [

        HumanMessage(content=input1),
        AIMessage(content=result1["output"]),
        AIMessage(content=intermediate_steps1)
    ]

)
# order here matters for some reason. If the order is changed, for the next question the tool will be invoke even though it is not needed

In [113]:
chat_history

[HumanMessage(content='how many aircraft are there in the sky currently?'),
 AIMessage(content='There are currently 4 aircraft in the sky. The aircraft IDs are: KL101, KL102, KL103, and BBB999.'),
 AIMessage(content='KL102\nAircraft ids: [\'KL101\', \'KL102\', \'KL103\', \'BBB999\']\nInfo on KL101 B74F index = 0\nPos: N49\'04\'7.0"  E006\'00\'2.0"\nHdg: 180   Trk: 180\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on KL102 B74F index = 1\nPos: N49\'21\'14.0"  E004\'21\'9.0"\nHdg: 210   Trk: 210\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on KL103 B74F index = 2\nPos: N50\'06\'37.0"  E003\'08\'47.0"\nHdg: 239   Trk: 239\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on BBB999 A320 index = 3\nPos: S14\'26\'56.0"  W013\'45\'17.0"\nHdg: 048   Trk: 048\nAlt: 15000 ft  V/S: 0 fpm\nCAS/TAS/GS: 150/188/188 kts   M: 0.300\n\n')]

In [118]:
input2 = "and which aircraft has the lowest altitude?"
result2 = agent_executor.invoke({"input": input2,
                      "chat_history": chat_history})



> Entering new AgentExecutor chain...


The aircraft with the lowest altitude is BBB999. It is currently at an altitude of 15000 ft.

> Finished chain.


In [121]:
try:
    intermediate_steps2 = result2['intermediate_steps'][0][1]
except IndexError:
    intermediate_steps2 = ''



In [122]:

chat_history.extend(
    [

        HumanMessage(content=input2),
        AIMessage(content=result2["output"]),
        AIMessage(content=intermediate_steps2)
    ]

)
# order here matters for some reason. If the order is changed, for the next question the tool will be invoke even though it is not needed

In [123]:
chat_history

[HumanMessage(content='how many aircraft are there in the sky currently?'),
 AIMessage(content='There are currently 4 aircraft in the sky. The aircraft IDs are: KL101, KL102, KL103, and BBB999.'),
 AIMessage(content='KL102\nAircraft ids: [\'KL101\', \'KL102\', \'KL103\', \'BBB999\']\nInfo on KL101 B74F index = 0\nPos: N49\'04\'7.0"  E006\'00\'2.0"\nHdg: 180   Trk: 180\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on KL102 B74F index = 1\nPos: N49\'21\'14.0"  E004\'21\'9.0"\nHdg: 210   Trk: 210\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on KL103 B74F index = 2\nPos: N50\'06\'37.0"  E003\'08\'47.0"\nHdg: 239   Trk: 239\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 340/485/485 kts   M: 0.805\n\nInfo on BBB999 A320 index = 3\nPos: S14\'26\'56.0"  W013\'45\'17.0"\nHdg: 048   Trk: 048\nAlt: 15000 ft  V/S: 0 fpm\nCAS/TAS/GS: 150/188/188 kts   M: 0.300\n\n'),
 HumanMessage(content='and which aircraft has the lowest altitude?'),
 AIMessage(

In [125]:
input3 = "What is the aircraft id with the lowest altitude?"
result3 = agent_executor.invoke({"input": input3,
                                 "chat_history": chat_history})



> Entering new AgentExecutor chain...


The aircraft ID with the lowest altitude is BBB999.

> Finished chain.


In [126]:
input4 = "What is the alititude of BBB999?"
result4 = agent_executor.invoke({"input": input4,
                                 "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_all_aircraft_info` with `{'command': 'GETACIDS'}`


The altitude of aircraft BBB999 is 15000 ft.

> Finished chain.


In [128]:
result4['intermediate_steps'][0][1]

''